In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from PIL import Image
import numpy as np
import pandas as pd

In [6]:
klasor_yolu = 'img_train/'  
dosya_isimleri = [dosya for dosya in os.listdir(klasor_yolu) if dosya.endswith('.jpg')]

hedef_boyut = (106, 106)

data = []
for dosya in dosya_isimleri:
    tam_yol = os.path.join(klasor_yolu, dosya)
    with Image.open(tam_yol) as img:
        img = img.convert('L').resize(hedef_boyut)  
        numpy_dizi = np.array(img).flatten() 
        data.append(numpy_dizi)

x_train = np.vstack(data)

csv_dosya_yolu = 'train_sol.csv' 
csv_matrisi = pd.read_csv(csv_dosya_yolu).values

y_train = csv_matrisi[:, 1:]



In [22]:
class VanillaNeuralNet(nn.Module):
    def __init__(self):
        super(VanillaNeuralNet, self).__init__()
        self.fc1 = nn.Linear(in_features=x_train.shape[1], out_features=128) 
        self.fc2 = nn.Linear(in_features=128, out_features=y_train.shape[1])   

    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x)          
        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = VanillaNeuralNet().to(device)


In [14]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

batch_size = 32 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
num_epochs = 100  
loss_values=[]
for epoch in range(num_epochs):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        labels=labels.float()
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss = running_loss / len(train_loader)
        loss_values.append(epoch_loss)

        running_loss += loss.item()
        if i % 200 == 199:    
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')
